In [1]:
import json
import pandas as pd
import re
import numpy as np
import os

In [40]:
annot_validated = pd.read_json("validation_out.json", lines=True)
annot_validated.head()

,user_idx,task_idx,task,tool_to_search,ok,better
0,185,0,Reviewing shipment schedules for the next quarter,Google Sheets,Y,-
1,185,1,Conducting cycle count on raw materials inventory,Google Sheets,Y,-
2,185,2,Analyzing sales data to identify trends in mat...,Google Sheets,Y,-
3,185,3,Creating and updating vendor contracts,Google Docs,Y,-
4,185,4,Verifying compliance with regulatory standards...,Confluence,Y,-


In [66]:
annot_validated['ok'].value_counts()

ok
Y    79835
N    24725
Name: count, dtype: int64

In [32]:
print(len(annot_validated))

# Which expected user_idx values are missing?
expected = set(range(annot_validated["user_idx"].min(), annot_validated["user_idx"].max() + 1)) 
present  = set(annot_validated["user_idx"].unique())
missing  = expected - present
print("missing user_idx:", sorted(missing))

# Verify each user_idx has exactly 10 task_idx values
counts = annot_validated.groupby("user_idx")["task_idx"].nunique()
bad    = counts[counts != 10]
print(f"\nusers with wrong task count: {len(bad)}")
display(bad.head())

104560
missing user_idx: []

users with wrong task count: 0


Series([], Name: task_idx, dtype: int64)

In [13]:
# random sample 50 rows of replaced annotations
for _, row in annot_validated[annot_validated['better'] != "-"].sample(50).iterrows():
    print(row['task'])
    print(row['tool_to_search'])
    print(row['better'])
    print('-' * 80)

Negotiating settlement agreement with disgruntled customer
Coda
Confluence
--------------------------------------------------------------------------------
Reviewing and updating financial reports
Confluence
Google Sheets
--------------------------------------------------------------------------------
Create and send welcome emails to new customers
Google Doc
Gmail
--------------------------------------------------------------------------------
Attend training session on new software update
Confluence
Google Slides
--------------------------------------------------------------------------------
Preparing contract amendments for approval from senior management
Confluence
Google Doc
--------------------------------------------------------------------------------
Prepare and submit compliance reports to management and regulatory bodies
Confluence
Google Doc
--------------------------------------------------------------------------------
Schedule maintenance window for new software release

In [41]:
# replace the annotations with the validated ones
annot_validated['final_tool'] = np.where(annot_validated['ok'] == 'Y', annot_validated['tool_to_search'], annot_validated['better'])
annot_validated.head()

,user_idx,task_idx,task,tool_to_search,ok,better,final_tool
0,185,0,Reviewing shipment schedules for the next quarter,Google Sheets,Y,-,Google Sheets
1,185,1,Conducting cycle count on raw materials inventory,Google Sheets,Y,-,Google Sheets
2,185,2,Analyzing sales data to identify trends in mat...,Google Sheets,Y,-,Google Sheets
3,185,3,Creating and updating vendor contracts,Google Docs,Y,-,Google Docs
4,185,4,Verifying compliance with regulatory standards...,Confluence,Y,-,Confluence


In [34]:
assert annot_validated['final_tool'].notna().all()
assert annot_validated['final_tool'].isin(['Coda', 'Confluence', 'Jira', 'Google Docs', 'Slack', 'Miro', 'Google Slides', 'Google Sheets', 'Gmail', 'GitHub']).all()

AssertionError: 

In [46]:
# fix some typos
annot_validated['final_tool'] = annot_validated['final_tool'].replace('Google Doc', 'Google Docs')
annot_validated['final_tool'] = annot_validated['final_tool'].apply(lambda x: ' '.join(word.capitalize() for word in x.split()))
annot_validated['final_tool'] = annot_validated['final_tool'].replace('Github', 'GitHub')


In [47]:
annot_validated[~annot_validated['final_tool'].isin(['Coda', 'Confluence', 'Jira', 'Google Docs', 'Slack', 'Miro', 'Google Slides', 'Google Sheets', 'Gmail', 'GitHub'])]

,user_idx,task_idx,task,tool_to_search,ok,better,final_tool
13444,9938,4,Update the CRM system with new contact informa...,"CRM System (not listed, use Confluence for gui...",Y,-,"Crm System (not Listed, Use Confluence For Gui..."
46225,5417,5,Analyzing candidate feedback to improve the in...,Google Forms,Y,-,Google Forms
73172,4389,2,Schedule appointments with new client,Google Calendar,Y,-,Google Calendar
73173,4389,3,Update operations team calendar,Google Calendar,Y,-,Google Calendar
73177,4389,7,Re-schedule meeting with conflicting attendees,Google Calendar,Y,-,Google Calendar
73179,4389,9,Book conference room reservations for internal...,Google Calendar,Y,-,Google Calendar
90915,385,5,Schedule meetings with stakeholders,Google Calendar,Y,-,Google Calendar
91406,2290,6,Coordinate team meeting to discuss current pro...,Google Calendar,Y,-,Google Calendar


In [55]:
# reannotate the miss-annotated ones
reannot = annot_validated[~annot_validated['final_tool'].isin(['Coda', 'Confluence', 'Jira', 'Google Docs', 'Slack', 'Miro', 'Google Slides', 'Google Sheets', 'Gmail', 'GitHub'])]

import json
from star_utils import query_llm_proxy, get_llm_proxy_api_token

# config ───────────────────────────────────
BACKEND   = "openai_direct_o3"  
TOKEN     = get_llm_proxy_api_token()
ALLOWED   = ["Coda","Confluence","Jira","Google Docs","Slack","Miro",
             "Google Slides","Google Sheets","Gmail","GitHub"]

schema = {
    "name": "tool_fix",
    "strict": True,
    "type":   "object",
    "properties": {
        "tool_validation": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {"tool_to_search": {"type": "string"}},
                "required": ["tool_to_search"],
                "additionalProperties": False
            },
            "minItems": 1,
            "maxItems": 1
        }
    },
    "required": ["tool_validation"],
    "additionalProperties": False
}


PROMPT_TMPL = """
You are validating **one** task–tool pair.

────────────────────────────────────────────────────────
TASK
{task}

ANNOTATED TOOL (invalid): {bad}

────────────────────────────────────────────────────────
ALLOWED TOOLS
  • Coda            → project docs, structured check-lists, trackers
  • Confluence      → wiki-style knowledge base, SOPs, policies
  • Jira            → task / ticket tracking, engineering back-log
  • Google Docs     → narrative docs, contracts, one-off write-ups
  • Slack           → real-time chat, quick Q&A, ad-hoc coordination
  • Miro            → brainstorming, white-boarding, visual planning
  • Google Slides   → presentations, slide decks
  • Google Sheets   → tabular data, metrics dashboards, trackers
  • Gmail           → external / formal email threads, 1-to-1 comms
  • GitHub          → code, version-controlled tech docs, CI logs
────────────────────────────────────────────────────────
DECISION RULES
1. **Pick exactly one tool from the allowed list** – nothing else.
2. Match the tool to the **primary work-mode** of the task:
   • Long-form policy / procedure → Confluence or Google Docs  
   • Ticket / bug / backlog item  → Jira  
   • Scheduling / coordination    → Slack or Gmail  
   • Metrics / tabular analysis    → Google Sheets  
   • Code / scripts / CI logs      → GitHub  
   • Brainstorm / mind-map         → Miro  
   • Slide deck prep               → Google Slides  
   • Multi-step check-lists        → Coda  
3. Only choose Confluence for wiki/knowledge-base work,
   not generic “find info” tasks.
4. If none of the mappings fit, pick the tool that minimizes
   search time given the task context.

────────────────────────────────────────────────────────
Return JSON only:
{{
  "tool_validation": [ {{ "tool_to_search": "<one allowed tool>" }} ]
}}
""".strip()


def ask_fix(task:str, bad:str)->str:
    prompt = PROMPT_TMPL.format(task=task, bad=bad, allowed=", ".join(ALLOWED))
    resp   = query_llm_proxy(prompt,
                             backend=BACKEND,
                             schema=schema,
                             sleep_time=0,
                             api_token=TOKEN)
    return json.loads(resp["chunk"]["text"])["tool_validation"][0]["tool_to_search"]

fixed_tools = []
for _, row in reannot.iterrows():
    try:
        new_tool = ask_fix(row["task"], row["tool_to_search"])
    except Exception as e:
        new_tool = "ERR"
        print(f"failed user_idx={row['user_idx']}, task_idx={row['task_idx']}  → {e}")
    fixed_tools.append(new_tool)

reannot = reannot.assign(fixed_tool=fixed_tools)
reannot

,user_idx,task_idx,task,tool_to_search,ok,better,final_tool,fixed_tool
13444,9938,4,Update the CRM system with new contact informa...,"CRM System (not listed, use Confluence for gui...",Y,-,"Crm System (not Listed, Use Confluence For Gui...",Jira
46225,5417,5,Analyzing candidate feedback to improve the in...,Google Forms,Y,-,Google Forms,Google Sheets
73172,4389,2,Schedule appointments with new client,Google Calendar,Y,-,Google Calendar,Gmail
73173,4389,3,Update operations team calendar,Google Calendar,Y,-,Google Calendar,Slack
73177,4389,7,Re-schedule meeting with conflicting attendees,Google Calendar,Y,-,Google Calendar,Slack
73179,4389,9,Book conference room reservations for internal...,Google Calendar,Y,-,Google Calendar,Slack
90915,385,5,Schedule meetings with stakeholders,Google Calendar,Y,-,Google Calendar,Slack
91406,2290,6,Coordinate team meeting to discuss current pro...,Google Calendar,Y,-,Google Calendar,Slack


In [56]:
# merge back the fixed tools
annot_validated['final_tool'].update(reannot['fixed_tool'])
assert annot_validated['final_tool'].isin(['Coda', 'Confluence', 'Jira', 'Google Docs', 'Slack', 'Miro', 'Google Slides', 'Google Sheets', 'Gmail', 'GitHub']).all()


In [58]:
# merge with the long queries dataset
data = []
with open("long_queries.jsonl", 'r', encoding='utf-8', errors='replace') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping invalid line: {e}")
            
df_queries = pd.DataFrame(data)

group_cols = [
    'name', 'employee_tenure', 'department', 'job_title',
    'company_size', 'company_sector', 'tasks'
]
df_queries['tasks'] = df_queries['tasks'].apply(tuple)
# Assign a unique group ID starting from 0
df_queries['user_idx'] = df_queries.groupby(group_cols).ngroup()
df_queries['tasks'] = df_queries['tasks'].apply(list)

keys = ['user_idx', 'task_idx']
df_queries = df_queries.merge(
    annot_validated[['user_idx', 'task_idx', 'final_tool']],
    on=keys,
    how='left'
)
df_queries.head()

Skipping invalid line: Extra data: line 1 column 1618 (char 1617)


,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task,user_idx,final_tool
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[resolution support customer service legal iss...,0,0,NaN,17,Gmail
1,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[client account payment issues legal billing d...,0,1,NaN,17,Confluence
2,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[escalate high priority cases to senior manage...,0,2,NaN,17,Slack
3,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[Drafting a document update response for clien...,0,3,NaN,17,Google Docs
4,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...","[Change of schedule coordination, Scheduling c...",0,4,NaN,17,Slack


In [59]:
len(df_queries)

104548

In [65]:
with open("long_queries_with_tools.json", "w", encoding="utf-8", errors="replace") as f:
    for record in df_queries.to_dict(orient="records"):
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

In [68]:
df_queries.explode('long_queries')

,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task,user_idx,final_tool
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",resolution support customer service legal issu...,0,0,NaN,17,Gmail
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",customer complaints email template resolution ...,0,0,NaN,17,Gmail
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",complaints management service customer support...,0,0,NaN,17,Gmail
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",legal firm customer service email responses co...,0,0,NaN,17,Gmail
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",mid-sized law firms complaint handling procedu...,0,0,NaN,17,Gmail
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104547,Liam,experienced,Operations,Production Scheduling Coordinator,large,Government & Public Sector,[Review and update production schedules for up...,public sector employee communication guidelines,10455,9,Communicate with teams and personnel regarding...,6408,Slack
104547,Liam,experienced,Operations,Production Scheduling Coordinator,large,Government & Public Sector,[Review and update production schedules for up...,government team updates coordination processes,10455,9,Communicate with teams and personnel regarding...,6408,Slack
104547,Liam,experienced,Operations,Production Scheduling Coordinator,large,Government & Public Sector,[Review and update production schedules for up...,scheduling coordinator changes email drafting,10455,9,Communicate with teams and personnel regarding...,6408,Slack
104547,Liam,experienced,Operations,Production Scheduling Coordinator,large,Government & Public Sector,[Review and update production schedules for up...,large company operations news dissemination,10455,9,Communicate with teams and personnel regarding...,6408,Slack
